In [1]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,LabelEncoder
from sklearn.metrics import accuracy_score,auc,roc_curve,precision_score,recall_score,f1_score
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
#LR,DTC,RF,GBDT,XGB,CAT,ETC,LGB,SVC,KNN,MLP,NB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import ExtraTreesClassifier ,StackingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
import random
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss
from sklearn.model_selection import KFold,cross_validate,LeaveOneOut,GridSearchCV,train_test_split,RandomizedSearchCV
import time
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics import roc_auc_score
import joblib
from joblib import Parallel,delayed
import sys
sys.path.append('F:\\my projects\\02.论文\\论文四_桥本甲状腺炎\\Project\\Codes\\202312_Hashimotos_thyroiditis\\05 modeling')
from Modeling_functions import *

In [2]:
X_train_scale,y_train,X_test_scale,y_test,= load_data()
print(X_train_scale.shape)
print(len(y_train))
print(X_test_scale.shape)
print(len(y_test))

(41, 79)
41
(20, 79)
20


In [3]:
random_state=42
k = X_train_scale.shape[0]
k

41

In [10]:

print('LR')
clf = LogisticRegression(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('RF')
clf = RandomForestClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('GBDT')
clf = GradientBoostingClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('ETC')
clf = ExtraTreesClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('SVM')
clf = SVC(random_state=random_state,probability=True)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('XGB')
clf = XGBClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('MLP')
clf = MLPClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('KNN')
clf = KNeighborsClassifier()
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('NB')
clf = GaussianNB()
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
print('CAT')
clf = CatBoostClassifier(random_state=random_state,verbose=False)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

LR
cv score:  0.9024390243902439
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.93
RF
cv score:  0.7073170731707317
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.91
GBDT
cv score:  0.5121951219512195
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.9000000000000001
ETC
cv score:  0.7317073170731707
train acc:  1.0
train auc:  1.0
test acc:  0.75
test auc:  0.9
SVM
cv score:  0.7804878048780488
train acc:  0.8048780487804879
train auc:  1.0
test acc:  0.7
test auc:  0.9
XGB
cv score:  0.7804878048780488
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.99
MLP
cv score:  0.9024390243902439
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.92
KNN
cv score:  0.8048780487804879
train acc:  0.8292682926829268
train auc:  0.9047619047619048
test acc:  0.65
test auc:  0.8200000000000001
NB
cv score:  0.7804878048780488
train acc:  0.8780487804878049
train auc:  0.875
test acc:  0.8
test auc:  0.78
CAT
cv score:  0.7073170731707317
t

In [8]:
def param_hyperopt(max_evals=3000):
    # 保存迭代过程
    trials = Trials()
    # 设置提前停止
    early_stop_fn = no_progress_loss(200)
    # 定义代理模型
    params_best = fmin(hyperopt_objective
                       , space=param_grid_simple
                       , algo=tpe.suggest
                       , max_evals=max_evals
                       , verbose=True
                       , trials=trials
                       , early_stop_fn=early_stop_fn
                       )
    # 打印最优参数，fmin会自动打印最佳分数
    print("\n", "\n", "best params: ", params_best,
          "\n")
    return params_best, trials

## LR

In [12]:
res = []
for C in np.arange(0.1, 2, 0.1):
    for class_weight in [None, 'balanced']:
        clf = LogisticRegression(penalty='l1'
                                 ,C =C
                                 ,solver='saga'
                                 ,class_weight=class_weight
                                 ,random_state=random_state)
        clf.fit(X_train_scale,y_train)
        res.append(clf.score(X_test_scale,y_test))
print(res)

[0.65, 0.65, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85]


In [13]:
res = []
for C in np.arange(0.1, 2, 0.1):
    for class_weight in [None, 'balanced']:
        for solver in ['lbfgs', 'newton-cg', 'sag', 'saga']: 
            clf = LogisticRegression(penalty='l2'
                                     ,C =C
                                     ,solver=solver
                                     ,class_weight=class_weight
                                     ,random_state=random_state)
            clf.fit(X_train_scale,y_train)
            res.append(clf.score(X_test_scale,y_test))
print(res)

[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.85, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.85, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.85, 0.85, 0.85, 0.9, 0.9, 0.85, 0.85, 0.9, 0.85, 0.85, 0.85, 0.9, 0.9, 0.9, 0.9, 0.9, 0.85, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.85, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.85]


In [14]:
res = []
for C in np.arange(0.1, 2, 0.1):
    for class_weight in [None, 'balanced']:
        for l1_ratio in np.arange(0.1, 1, 0.1): 
            clf = LogisticRegression(penalty='elasticnet'
                                     ,C =C
                                     ,solver='saga'
                                     ,l1_ratio=l1_ratio
                                     ,class_weight=class_weight
                                     ,random_state=random_state)
            clf.fit(X_train_scale,y_train)
            res.append(clf.score(X_test_scale,y_test))
print(res)

[0.85, 0.8, 0.8, 0.7, 0.7, 0.65, 0.65, 0.65, 0.65, 0.85, 0.8, 0.8, 0.7, 0.7, 0.65, 0.65, 0.65, 0.65, 0.85, 0.85, 0.85, 0.8, 0.8, 0.8, 0.75, 0.7, 0.7, 0.85, 0.85, 0.85, 0.85, 0.8, 0.8, 0.75, 0.7, 0.7, 0.85, 0.85, 0.85, 0.85, 0.85, 0.8, 0.8, 0.8, 0.8, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.8, 0.8, 0.8, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.8, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.8, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,

In [9]:
param_grid = [
    { 'penalty': ['l1'], 'C': np.arange(0.1, 2, 0.1).tolist(), 'solver': ['saga'],'class_weight':[None, 'balanced']}, 
    { 'penalty': ['l2'], 'C': np.arange(0.1, 2, 0.1).tolist(), 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],'class_weight':[None, 'balanced']},
    {'penalty': ['elasticnet'], 'C': np.arange(0.1, 2, 0.1).tolist(), 'l1_ratio': np.arange(0.1, 1, 0.1).tolist(), 'solver': ['saga'],'class_weight':[None, 'balanced']}
]
clf = LogisticRegression(random_state=random_state)
cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
search = GridSearchCV(estimator=clf
                     ,param_grid=param_grid
                     ,scoring = 
                     ,verbose = True
                     ,cv = cv
                    )
search.fit(X_train_scale,y_train)
print(search.best_params_)
print(search.best_score_)
print(search.best_estimator_)


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:
clf_best = LogisticRegression(C=0.30000000000000004, class_weight='balanced',
                   random_state=42, solver='saga')
clf_res(clf_best,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.9024390243902439
train acc:  0.975609756097561
train auc:  1.0
test acc:  0.9
test auc:  0.9


In [11]:
joblib.dump(clf_best,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/LR_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/LR_best_l1.dat']

## RF

In [15]:
clf = RandomForestClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.7073170731707317
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.91


In [40]:
param_grid_simple={'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"criterion": hp.choice("criterion",["gini","entropy"])
                    ,"max_depth": hp.quniform("max_depth",1,10,1)
                    ,'min_samples_split':hp.quniform("min_samples_split",2,5,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,5,1)
                    ,"max_samples": hp.quniform("max_samples",0.5,1,0.05)
                    ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.01)
                     }
def hyperopt_objective(params):
    clf = RandomForestClassifier(n_estimators = int(params["n_estimators"])
                                  ,criterion = params["criterion"]
                                  ,max_depth = int(params["max_depth"])
                                 ,min_samples_split = int(params['min_samples_split'])
                                 ,min_samples_leaf = int(params['min_samples_leaf'])
                                  ,max_features = params['max_features']
                                  ,max_samples = params["max_samples"]
                                  ,min_impurity_decrease = params["min_impurity_decrease"]
                                  ,random_state=random_state
                                  ,verbose=False
                                )
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # res = cross_validate(clf
    #                     ,X=X_train_scale
    #                     ,y=y_train
    #                     ,cv=cv
    #                     ,scoring='accuracy') 
    # return -np.mean(res['test_score'])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    return -res

In [42]:
params_best_acc1, trials = param_hyperopt()

  7%|▋         | 217/3000 [00:35<07:29,  6.20trial/s, best loss: -0.85]

 
 best params:  {'criterion': 1, 'max_depth': 3.0, 'max_features': 0.6000000000000001, 'max_samples': 0.9500000000000001, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 1.0, 'min_samples_split': 3.0, 'n_estimators': 150.0} 


In [18]:
clf1 = RandomForestClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc1["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc1["criterion"])]
                                  ,max_depth = int(params_best_acc1["max_depth"])
                                 ,min_samples_split = int(params_best_acc1['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc1['min_samples_leaf'])
                                  ,max_features = params_best_acc1['max_features']
                                  ,max_samples = params_best_acc1["max_samples"]
                                  ,min_impurity_decrease = params_best_acc1["min_impurity_decrease"]
                                  ,verbose=False)
clf_res(clf1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)


cv score:  0.7804878048780488
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.9400000000000001


In [19]:
params_best_acc2, trials = param_hyperopt()
clf2 = RandomForestClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc2["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc2["criterion"])]
                                  ,max_depth = int(params_best_acc2["max_depth"])
                                 ,min_samples_split = int(params_best_acc2['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc2['min_samples_leaf'])
                                  ,max_features = params_best_acc2['max_features']
                                  ,max_samples = params_best_acc2["max_samples"]
                                  ,min_impurity_decrease = params_best_acc2["min_impurity_decrease"]
                                  ,verbose=False)
clf_res(clf2,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

  8%|▊         | 253/3000 [00:49<08:59,  5.09trial/s, best loss: -0.85]

 
 best params:  {'criterion': 1, 'max_depth': 3.0, 'max_features': 0.8500000000000001, 'max_samples': 0.9, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 1.0, 'min_samples_split': 5.0, 'n_estimators': 220.0} 
cv score:  0.7804878048780488
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.97


In [20]:
param_grid_simple={'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"criterion": hp.choice("criterion",["gini","entropy"])
                    ,"max_depth": hp.quniform("max_depth",1,5,1)
                    ,'min_samples_split':hp.quniform("min_samples_split",2,5,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,8,1)
                    ,"max_samples": hp.quniform("max_samples",0.5,1,0.05)
                    ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.05)
                     }
def hyperopt_objective(params):
    clf = RandomForestClassifier(n_estimators = int(params["n_estimators"])
                                  ,criterion = params["criterion"]
                                  ,max_depth = int(params["max_depth"])
                                 ,min_samples_split = int(params['min_samples_split'])
                                 ,min_samples_leaf = int(params['min_samples_leaf'])
                                  ,max_features = params['max_features']
                                  ,max_samples = params["max_samples"]
                                  ,min_impurity_decrease = params["min_impurity_decrease"]
                                  ,random_state=random_state
                                  ,verbose=False
                                )
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # res = cross_validate(clf
    #                     ,X=X_train_scale
    #                     ,y=y_train
    #                     ,cv=cv
    #                     ,scoring='accuracy') 
    # return -np.mean(res['test_score'])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    return -res
params_best_acc3, trials = param_hyperopt()
clf3 = RandomForestClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc3["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc3["criterion"])]
                                  ,max_depth = int(params_best_acc3["max_depth"])
                                 ,min_samples_split = int(params_best_acc3['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc3['min_samples_leaf'])
                                  ,max_features = params_best_acc3['max_features']
                                  ,max_samples = params_best_acc3["max_samples"]
                                  ,min_impurity_decrease = params_best_acc3["min_impurity_decrease"]
                                  ,verbose=False)
clf_res(clf3,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

  8%|▊         | 226/3000 [00:33<06:48,  6.78trial/s, best loss: -0.85]

 
 best params:  {'criterion': 0, 'max_depth': 5.0, 'max_features': 1.0, 'max_samples': 0.9500000000000001, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1.0, 'min_samples_split': 4.0, 'n_estimators': 180.0} 
cv score:  0.7317073170731707
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.9600000000000001


In [21]:
param_grid_simple={'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"criterion": hp.choice("criterion",["gini","entropy"])
                    ,"max_depth": hp.quniform("max_depth",1,8,1)
                    ,'min_samples_split':hp.quniform("min_samples_split",2,5,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,8,1)
                    ,"max_samples": hp.quniform("max_samples",0.5,1,0.05)
                    ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.05)
                     }
params_best_acc4, trials = param_hyperopt()
clf4 = RandomForestClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc4["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc4["criterion"])]
                                  ,max_depth = int(params_best_acc4["max_depth"])
                                 ,min_samples_split = int(params_best_acc4['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc4['min_samples_leaf'])
                                  ,max_features = params_best_acc4['max_features']
                                  ,max_samples = params_best_acc4["max_samples"]
                                  ,min_impurity_decrease = params_best_acc4["min_impurity_decrease"]
                                  ,verbose=False)
clf_res(clf4,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

 11%|█         | 327/3000 [01:05<08:54,  5.00trial/s, best loss: -0.85]

 
 best params:  {'criterion': 0, 'max_depth': 5.0, 'max_features': 0.9, 'max_samples': 0.8500000000000001, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1.0, 'min_samples_split': 2.0, 'n_estimators': 290.0} 
cv score:  0.7317073170731707
train acc:  1.0
train auc:  1.0
test acc:  0.85
test auc:  0.9449999999999998


In [22]:
joblib.dump(clf2,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/RF_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/RF_best_l1.dat']

In [35]:
param_grid_simple={"criterion": hp.choice("criterion",["gini","entropy"])
                                ,"max_depth": hp.quniform("max_depth",1,10,1)
                                ,'min_samples_split':hp.quniform("min_samples_split",2,5,1)
                                ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,10,1)
                                ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                                ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.01)
                                 }
def hyperopt_objective(params):
    clf = DecisionTreeClassifier(criterion = params["criterion"]
                                          ,max_depth = int(params["max_depth"])
                                         ,min_samples_split = int(params['min_samples_split'])
                                         ,min_samples_leaf = int(params['min_samples_leaf'])
                                          ,max_features = params["max_features"]
                                          ,min_impurity_decrease = params["min_impurity_decrease"]
                                          ,random_state=random_state)
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # res = cross_validate(clf
    #                     ,X=X_train
    #                     ,y=y_train
    #                     ,cv=cv
    #                     ,n_jobs=16
    #                     ,scoring='accuracy')
    # return -np.mean(res['test_score'])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    # res2 = clf.score(X_test2,y_test2)
    # res = (res3+res2)/2
    return -res

In [36]:
params_best_acc1, trials = param_hyperopt()

 11%|█         | 317/3000 [00:05<00:42, 62.87trial/s, best loss: -0.9]  

 
 best params:  {'criterion': 1, 'max_depth': 6.0, 'max_features': 0.8, 'min_impurity_decrease': 0.08, 'min_samples_leaf': 2.0, 'min_samples_split': 2.0} 


In [38]:
clf_acc1 = DecisionTreeClassifier(criterion = ["gini","entropy"][int(params_best_acc1["criterion"])]
                                  ,max_depth = int(params_best_acc1["max_depth"])
                                 ,min_samples_split = int(params_best_acc1['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc1['min_samples_leaf'])
                                  ,max_features = params_best_acc1["max_features"]
                                  ,min_impurity_decrease = params_best_acc1["min_impurity_decrease"]
                                 ,random_state=random_state)
clf_res(clf_acc1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.7073170731707317
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.9


In [39]:
joblib.dump(clf_acc1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/DTC_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/DTC_best_l1.dat']

## SVM

In [30]:
param_grid_simple={'C': hp.quniform("C",0.05,2,0.05)
                              ,"kernel": hp.choice("kernel",["rbf",'linear'])
                   ,'class_weight':hp.choice('class_weight',[None, 'balanced'])
                             ,"gamma" :hp.quniform("gamma",0.05,2,0.05)
                                                   }
def hyperopt_objective(params):
    clf = SVC(C = params["C"]
              ,kernel = params["kernel"]
              ,gamma=params['gamma']
              #,degree=int(params['degree'])
              ,class_weight=params['class_weight']
                ,probability=True
              ,random_state=random_state
              ,verbose=False)
    cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    res = cross_validate(clf
                        ,X=X_train_scale
                        ,y=y_train
                        ,cv=cv
                        ,scoring='accuracy') 
    return -np.mean(res['test_score'])

In [31]:
params_best_acc1, trials = param_hyperopt()

  7%|▋         | 202/3000 [00:10<02:23, 19.49trial/s, best loss: -0.9024390243902439]

 
 best params:  {'C': 1.3, 'class_weight': 1, 'gamma': 1.5, 'kernel': 1} 


In [32]:
clf1 = SVC(C = params_best_acc1["C"]
              ,kernel = ["rbf",'linear'][int(params_best_acc1["kernel"])]
              ,gamma=params_best_acc1['gamma']
              #,degree=int(params['degree'])
              ,class_weight=[None, 'balanced'][int(params_best_acc1['class_weight'])]
                ,probability=True
              ,random_state=random_state
              ,verbose=False)
clf_res(clf1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.9024390243902439
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.96


In [44]:
joblib.dump(clf1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/SVC_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/SVC_best_l1.dat']

## GBDT

In [53]:
param_grid_simple={"learning_rate": hp.quniform("learning_rate",0.001,0.2,0.001)
                    ,'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"subsample": hp.quniform("subsample",0.5,1,0.05)
                      ,"criterion": hp.choice("criterion",['friedman_mse', 'squared_error'])
                    ,'min_samples_split':hp.quniform("min_samples_split",2,10,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,5,1)
                      ,"max_depth": hp.quniform("max_depth",1,6,1)
                       ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                      ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.01)
                                                       }

In [54]:
def hyperopt_objective(params):
    clf = GradientBoostingClassifier(n_estimators = int(params["n_estimators"])
                                  ,learning_rate = params["learning_rate"]
                                  ,criterion = params["criterion"]
                                     ,min_samples_split =  int(params['min_samples_split'])
                                     ,min_samples_leaf =  int(params['min_samples_leaf'])
                                  ,max_depth = int(params["max_depth"])
                                  ,max_features = params["max_features"]
                                  ,subsample = params["subsample"]
                                  ,min_impurity_decrease = params["min_impurity_decrease"]
                                  ,random_state=random_state
                                  ,verbose=False)
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # res = cross_validate(clf
    #                     ,X=X_train
    #                     ,y=y_train
    #                     ,cv=cv
    #                     ,n_jobs=16
    #                     ,scoring='accuracy') 
    # return -np.mean(res['test_score'])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    # res2 = clf.score(X_test2,y_test2)
    # res = (res3+res2)/2
    return -res

In [55]:
params_best_acc1, trials = param_hyperopt()

  7%|▋         | 217/3000 [00:22<04:54,  9.46trial/s, best loss: -0.95]

 
 best params:  {'criterion': 0, 'learning_rate': 0.047, 'max_depth': 4.0, 'max_features': 0.8, 'min_impurity_decrease': 0.21, 'min_samples_leaf': 3.0, 'min_samples_split': 4.0, 'n_estimators': 160.0, 'subsample': 1.0} 


In [56]:
clf_acc1 = GradientBoostingClassifier(n_estimators = int(params_best_acc1["n_estimators"])
                                  ,learning_rate = params_best_acc1["learning_rate"]
                                  ,criterion = ['friedman_mse', 'squared_error'][int(params_best_acc1["criterion"])]
                                     ,min_samples_split =  int(params_best_acc1['min_samples_split'])
                                     ,min_samples_leaf =  int(params_best_acc1['min_samples_leaf'])
                                  ,max_depth = int(params_best_acc1["max_depth"])
                                  ,max_features = params_best_acc1["max_features"]
                                  ,subsample = params_best_acc1["subsample"]
                                  ,min_impurity_decrease = params_best_acc1["min_impurity_decrease"]
                                  ,random_state=random_state
                                  ,verbose=False)
clf_res(clf_acc1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.6585365853658537
train acc:  1.0
train auc:  1.0
test acc:  0.95
test auc:  0.9600000000000001


In [57]:
joblib.dump(clf_acc1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/GBDT_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/GBDT_best_l1.dat']

In [58]:
joblib.load('F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/GBDT_best_l1.dat')

GradientBoostingClassifier(learning_rate=0.047, max_depth=4, max_features=0.8,
                           min_impurity_decrease=0.21, min_samples_leaf=3,
                           min_samples_split=4, n_estimators=160,
                           random_state=42, verbose=False)

## ETC

In [60]:
param_grid_simple={'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"criterion": hp.choice("criterion",["gini","entropy"])
                    ,"max_depth": hp.quniform("max_depth",1,5,1)
                    ,'min_samples_split':hp.quniform("min_samples_split",2,5,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,5,1)
                    ,"max_samples": hp.quniform("max_samples",0.5,1,0.05)
                    ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.05)
                     }
def hyperopt_objective(params):
    clf = ExtraTreesClassifier(n_estimators = int(params["n_estimators"])
                                  ,criterion = params["criterion"]
                                  ,max_depth = int(params["max_depth"])
                                 ,min_samples_split = int(params['min_samples_split'])
                                 ,min_samples_leaf = int(params['min_samples_leaf'])
                                  ,max_features = params['max_features']
                                  ,max_samples = params["max_samples"]
                                  ,min_impurity_decrease = params["min_impurity_decrease"]
                                  ,random_state=random_state
                               ,bootstrap=True
                                  ,verbose=False
                                )
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    # res2 = clf.score(X_test2,y_test2)
    # res = (res3+res2)/2
    return -res

In [61]:
params_best_acc1, trials = param_hyperopt()

 10%|█         | 310/3000 [00:38<05:32,  8.09trial/s, best loss: -0.9] 

 
 best params:  {'criterion': 1, 'max_depth': 5.0, 'max_features': 0.8, 'max_samples': 0.75, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 2.0, 'min_samples_split': 5.0, 'n_estimators': 100.0} 


In [62]:
clf_acc1 = ExtraTreesClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc1["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc1["criterion"])]
                                  ,max_depth = int(params_best_acc1["max_depth"])
                                 ,min_samples_split = int(params_best_acc1['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc1['min_samples_leaf'])
                                  ,max_features = params_best_acc1['max_features']
                                  ,max_samples = params_best_acc1["max_samples"]
                                  ,min_impurity_decrease = params_best_acc1["min_impurity_decrease"]
                                ,bootstrap=True
                                  ,verbose=False)
clf_res(clf_acc1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.7317073170731707
train acc:  0.975609756097561
train auc:  1.0
test acc:  0.9
test auc:  0.92


In [63]:
param_grid_simple={'n_estimators': hp.quniform("n_estimators",100,300,10)
                    ,"criterion": hp.choice("criterion",["gini","entropy"])
                    ,"max_depth": hp.quniform("max_depth",1,10,1)
                    ,'min_samples_split':hp.quniform("min_samples_split",2,10,1)
                    ,'min_samples_leaf':hp.quniform('min_samples_leaf',1,5,1)
                    ,"max_samples": hp.quniform("max_samples",0.5,1,0.05)
                    ,"max_features": hp.quniform("max_features",0.5,1,0.05)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1,0.05)
                     }

In [64]:
params_best_acc2, trials = param_hyperopt()

 10%|▉         | 299/3000 [00:37<05:35,  8.05trial/s, best loss: -0.9]

 
 best params:  {'criterion': 0, 'max_depth': 10.0, 'max_features': 0.8, 'max_samples': 0.75, 'min_impurity_decrease': 0.05, 'min_samples_leaf': 1.0, 'min_samples_split': 8.0, 'n_estimators': 110.0} 


In [65]:
clf_acc2 = ExtraTreesClassifier(random_state=random_state
                                ,n_estimators = int(params_best_acc2["n_estimators"])
                                  ,criterion = ["gini","entropy"][int(params_best_acc2["criterion"])]
                                  ,max_depth = int(params_best_acc2["max_depth"])
                                 ,min_samples_split = int(params_best_acc2['min_samples_split'])
                                 ,min_samples_leaf = int(params_best_acc2['min_samples_leaf'])
                                  ,max_features = params_best_acc2['max_features']
                                  ,max_samples = params_best_acc2["max_samples"]
                                  ,min_impurity_decrease = params_best_acc2["min_impurity_decrease"]
                                ,bootstrap=True
                                  ,verbose=False)
clf_res(clf_acc2,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.7317073170731707
train acc:  0.975609756097561
train auc:  1.0
test acc:  0.9
test auc:  0.9


In [66]:
joblib.dump(clf_acc1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/ETC_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/ETC_best_l1.dat']

In [67]:
joblib.load('F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/ETC_best_l1.dat')

ExtraTreesClassifier(bootstrap=True, criterion='entropy', max_depth=5,
                     max_features=0.8, max_samples=0.75,
                     min_impurity_decrease=0.1, min_samples_leaf=2,
                     min_samples_split=5, random_state=42, verbose=False)

## MLP

In [69]:
param_grid_simple={'activation':hp.choice('activation',['logistic', 'tanh', 'relu'])
                   ,'hidden_layer_sizes':hp.quniform('hidden_layer_sizes',100,200,10)
                                ,'alpha':hp.quniform('alpha',0.0001,0.05,0.0005)
                                # ,'max_iter':hp.quniform('max_iter',200,500,10)
                                }

In [70]:
def hyperopt_objective(params):
    clf =MLPClassifier(activation=params['activation']
                       ,alpha = params['alpha']
                       ,hidden_layer_sizes=(int(params['hidden_layer_sizes']),)
                       ,max_iter = 1000
                      ,random_state=random_state
                            ,verbose=False)
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # validation_loss = cross_validate(clf,X=X_train,y=y_train
    #                                  ,scoring='accuracy'
    #                                  ,cv=cv
    #                                  ,verbose=False
    #                                  ,n_jobs=-1
    # 
    #                                 )
    # return -np.mean(validation_loss["test_score"])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    # res2 = clf.score(X_test2,y_test2)
    # res = (res3+res2)/2
    return -res

In [71]:
params_best_acc1, trials = param_hyperopt()

  7%|▋         | 205/3000 [00:28<06:24,  7.26trial/s, best loss: -0.9]

 
 best params:  {'activation': 2, 'alpha': 0.0, 'hidden_layer_sizes': 190.0, 'max_iter': 220.0} 


In [72]:
clf_acc1 = MLPClassifier(activation=['identity', 'logistic', 'tanh', 'relu'][int(params_best_acc1['activation'])]
                       ,alpha = params_best_acc1['alpha']
                         ,hidden_layer_sizes=int(params_best_acc1['hidden_layer_sizes'])
                       ,max_iter = 1000
                      ,random_state=random_state
                            ,verbose=False)
clf_res(clf_acc1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.9024390243902439
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.9500000000000001


In [73]:
joblib.dump(clf_acc1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/MLP_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/MLP_best_l1.dat']

In [74]:
joblib.load('F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/MLP_best_l1.dat')

MLPClassifier(activation='tanh', alpha=0.0, hidden_layer_sizes=190,
              max_iter=1000, random_state=42)

## KNN

In [75]:
KNeighborsClassifier?

In [76]:
param_grid_simple={'n_neighbors':hp.quniform('n_neighbors',2,10,1)
                   ,'weights':hp.choice('weights',['uniform', 'distance'])
                   ,'p':hp.choice('p',[1, 2])
                                }

In [77]:
def hyperopt_objective(params):
    clf =KNeighborsClassifier(n_neighbors=int(params['n_neighbors'])
                   ,weights=params['weights']
                   ,p=params['p'])
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # validation_loss = cross_validate(clf,X=X_train,y=y_train
    #                                  ,scoring='accuracy'
    #                                  ,cv=cv
    #                                  ,verbose=False
    #                                  ,n_jobs=-1
    # 
    #                                 )
    # return -np.mean(validation_loss["test_score"])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    return -res

In [78]:
params_best_acc1, trials = param_hyperopt()

  9%|▉         | 264/3000 [00:01<00:20, 135.94trial/s, best loss: -0.9]

 
 best params:  {'n_neighbors': 2.0, 'p': 1, 'weights': 0} 


In [83]:
clf_acc1 = KNeighborsClassifier(n_neighbors=int(params_best_acc1['n_neighbors'])
                   ,weights=['uniform', 'distance'][int(params_best_acc1['weights'])]
                   ,p=[1, 2][int(params_best_acc1['p'])])
clf_res(clf_acc1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.6829268292682927
train acc:  0.926829268292683
train auc:  0.9714285714285714
test acc:  0.9
test auc:  0.95


In [84]:
joblib.dump(clf_acc1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/KNN_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/KNN_best_l1.dat']

## XGB

In [85]:
clf = XGBClassifier(random_state=random_state)
clf_res(clf,X_train_scale,y_train,X_test_scale,y_test,k,random_state)

cv score:  0.7804878048780488
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.99


In [86]:
joblib.dump(clf,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/XGB_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/XGB_best_l1.dat']

In [5]:
CatBoostClassifier?

In [13]:
param_grid_simple={'iterations':hp.quniform('iterations',500,700,10)
                   ,'learning_rate':hp.quniform('learning_rate',0.005,0.05,0.005)
                   ,'depth':hp.quniform('depth',1,16,1)
                   ,'l2_leaf_reg':hp.quniform('l2_leaf_reg',0,10,1)
                   ,'model_size_reg':hp.quniform('model_size_reg',0.001,2,0.005)
                   ,'rsm':hp.quniform('rsm',0.001,1,0.005)
                   ,'feature_border_type':hp.choice('feature_border_type',['Median','Uniform','UniformAndQuantiles','GreedyLogSum','MaxLogSum','MinEntropy'])
                   ,'bagging_temperature':hp.quniform('bagging_temperature',0,1,0.05)
                   ,'subsample':hp.quniform('subsample',0.5,1,0.05)
                   ,'min_data_in_leaf':hp.quniform('min_data_in_leaf',1,5,1)
                                }

In [14]:
def hyperopt_objective(params):
    clf =CatBoostClassifier(
                            iterations=int(params['iterations'])
                            ,learning_rate=params['learning_rate']
                            ,depth=int(params['depth'])
                            ,l2_leaf_reg=params['l2_leaf_reg']
                            ,model_size_reg=params['model_size_reg']
                            ,rsm=params['rsm']
                            ,feature_border_type=params['feature_border_type']
                            ,bagging_temperature=params['bagging_temperature']
                            ,subsample=params['subsample']
                            ,min_data_in_leaf=int(params['min_data_in_leaf'])
                            ,random_seed=random_state
                            ,verbose=False
                            )
   
    # cv = KFold(n_splits=k,shuffle=True,random_state=random_state)
    # validation_loss = cross_validate(clf,X=X_train,y=y_train
    #                                  ,scoring='accuracy'
    #                                  ,cv=cv
    #                                  ,verbose=False
    #                                  ,n_jobs=-1
    # 
    #                                 )
    # return -np.mean(validation_loss["test_score"])
    clf.fit(X_train_scale,y_train)
    res = clf.score(X_test_scale,y_test)
    return -res

In [15]:
params_best_acc1, trials = param_hyperopt()

  7%|▋         | 200/3000 [10:43<2:30:07,  3.22s/trial, best loss: -0.9] 

 
 best params:  {'bagging_temperature': 0.8500000000000001, 'depth': 3.0, 'feature_border_type': 0, 'iterations': 530.0, 'l2_leaf_reg': 1.0, 'learning_rate': 0.01, 'min_data_in_leaf': 2.0, 'model_size_reg': 0.5750000000000001, 'rsm': 0.7000000000000001, 'subsample': 1.0} 


In [16]:
clf1 =CatBoostClassifier(
                    iterations=int(params_best_acc1['iterations'])
                    ,learning_rate=params_best_acc1['learning_rate']
                    ,depth=int(params_best_acc1['depth'])
                    ,l2_leaf_reg=params_best_acc1['l2_leaf_reg']
                    ,model_size_reg=params_best_acc1['model_size_reg']
                    ,rsm=params_best_acc1['rsm']
                    ,feature_border_type=['Median','Uniform','UniformAndQuantiles','GreedyLogSum','MaxLogSum','MinEntropy'][int(params_best_acc1['feature_border_type'])]
                    ,bagging_temperature=params_best_acc1['bagging_temperature']
                    ,subsample=params_best_acc1['subsample']
                    ,min_data_in_leaf=int(params_best_acc1['min_data_in_leaf'])
                    ,random_seed=random_state
                    ,verbose=False
                    
                    )
clf_res(clf1,X_train_scale,y_train,X_test_scale,y_test,k,random_state)
                    

cv score:  0.7560975609756098
train acc:  1.0
train auc:  1.0
test acc:  0.9
test auc:  0.96


In [17]:
joblib.dump(clf1,'F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/CAT_best_l1.dat')

['F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/07.Modeling/Layer1/CAT_best_l1.dat']